# Problem Set 2

In [3]:
run(`ls`)

PS2_solution.ipynb


Process(`ls`, ProcessExited(0))

In [4]:
using Parameters, Plots, DiscreteMarkovChains
using DelimitedFiles, BenchmarkTools
using DataFrames, CSV

# I'll remove my name if this is the final file
include("../JuliaCode/Hugget_model_mitch.jl") 

solve_the_model (generic function with 4 methods)

## Question I

## Question II

## Question III